<a href="https://colab.research.google.com/github/tomazcavalcantee/IDOR/blob/main/Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# ============================================================
# 1) Ler TXT (Track_OMC_0001)
# ============================================================
def read_track_omc_txt(path, sep="\t"):
    """
    Lê um .txt/.tsv exportado pelo sistema (colunas com FP1.ForZ, FP2.ForZ etc).
    """
    df = pd.read_csv(path, sep=sep, engine="python")
    df.columns = [c.strip() for c in df.columns]  # tira espaços extras

    # tempo (segundos)
    t = pd.to_numeric(df["TimeStamp"], errors="coerce").to_numpy()
    t = t - np.nanmin(t)

    return df, t


# ============================================================
# 2) Utilitários: fill NaN + filtro simples (média móvel)
# ============================================================
def fill_nans_1d(x):
    x = np.asarray(x, dtype=float)
    idx = np.arange(len(x))
    good = np.isfinite(x)
    if good.sum() < 2:
        return x
    y = x.copy()
    y[~good] = np.interp(idx[~good], idx[good], x[good])
    return y

def moving_average(x, win):
    """
    média móvel simples (win em amostras). win >= 1
    """
    x = np.asarray(x, dtype=float)
    if win <= 1:
        return x
    kernel = np.ones(win) / win
    return np.convolve(x, kernel, mode="same")


# ============================================================
# 3) Detectar HS e TO a partir de ForZ (positiva)
# ============================================================
def detect_hs_to_from_fz(t, fz, thr_N=40.0, smooth_s=0.02, min_contact_s=0.08):
    """
    HS = cruzamento ascendente de fz > thr
    TO = cruzamento descendente de fz < thr

    Parâmetros:
      thr_N: limiar em Newtons
      smooth_s: suavização (segundos) por média móvel
      min_contact_s: remove contatos curtos (segundos) = ruído

    Retorna:
      hs_idx, to_idx (índices), hs_t, to_t, fz_s (sinal filtrado)
    """
    t = np.asarray(t, float)
    fz = fill_nans_1d(fz)

    fs = 1.0 / np.nanmean(np.diff(t))
    win = int(round(smooth_s * fs))
    win = max(win, 1)
    fz_s = moving_average(fz, win)

    contact = fz_s > thr_N
    # transições
    d = np.diff(contact.astype(int))
    hs_idx = np.where(d == 1)[0] + 1   # False->True
    to_idx = np.where(d == -1)[0] + 1  # True->False

    # garantir pares HS->TO
    if len(hs_idx) == 0 or len(to_idx) == 0:
        return np.array([], int), np.array([], int), np.array([]), np.array([]), fz_s

    # se começa em contato, descarta TO inicial até ter HS antes
    if to_idx[0] < hs_idx[0]:
        to_idx = to_idx[1:]
    # se termina em contato, descarta HS final sem TO
    if len(hs_idx) > 0 and len(to_idx) > 0 and hs_idx[-1] > to_idx[-1]:
        hs_idx = hs_idx[:-1]

    n = min(len(hs_idx), len(to_idx))
    hs_idx, to_idx = hs_idx[:n], to_idx[:n]

    # remover contatos curtos
    min_samples = int(round(min_contact_s * fs))
    keep = (to_idx - hs_idx) >= min_samples
    hs_idx = hs_idx[keep]
    to_idx = to_idx[keep]

    return hs_idx, to_idx, t[hs_idx], t[to_idx], fz_s


# ============================================================
# 4) Detectar "gaps": trechos sem contato em FP1 e FP2
# ============================================================
def detect_no_contact_gaps(t, fz1_s, fz2_s, thr_N=40.0, min_gap_s=0.25):
    """
    Gap = (FP1 <= thr) AND (FP2 <= thr) por um tempo mínimo.
    Retorna lista de (t_ini, t_fim) e máscara booleana gap_mask.
    """
    t = np.asarray(t, float)
    fs = 1.0 / np.nanmean(np.diff(t))

    no_contact = (fz1_s <= thr_N) & (fz2_s <= thr_N)

    # achar segmentos contínuos True
    idx = np.where(no_contact)[0]
    if len(idx) == 0:
        return [], no_contact

    # segmentação por quebras
    breaks = np.where(np.diff(idx) > 1)[0]
    starts = np.r_[idx[0], idx[breaks + 1]]
    ends   = np.r_[idx[breaks], idx[-1]]

    gaps = []
    for s, e in zip(starts, ends):
        if (e - s + 1) >= int(round(min_gap_s * fs)):
            gaps.append((t[s], t[e]))

    return gaps, no_contact


# ============================================================
# 5) Limpeza global de eventos (para todos)
# ============================================================
def clean_events_global(t, hs_t, to_t, min_step_s=0.35, max_step_s=2.0):
    """
    Remove eventos com tempos inconsistentes.
    Regras:
      - contato precisa ter TO > HS
      - duração do contato entre min/max (aqui usamos max_step_s como teto geral)
      - HS consecutivos devem respeitar min_step_s
    Retorna hs_t2, to_t2
    """
    hs_t = np.asarray(hs_t, float)
    to_t = np.asarray(to_t, float)

    if len(hs_t) == 0 or len(to_t) == 0:
        return hs_t, to_t

    # 1) TO depois de HS
    keep = to_t > hs_t
    hs_t, to_t = hs_t[keep], to_t[keep]

    if len(hs_t) < 2:
        return hs_t, to_t

    # 2) duração de contato plausível (opcional: aqui bem “larga”)
    contact_dur = to_t - hs_t
    keep = (contact_dur > 0.05) & (contact_dur < max_step_s)
    hs_t, to_t = hs_t[keep], to_t[keep]

    if len(hs_t) < 2:
        return hs_t, to_t

    # 3) remover HS muito próximos (ruído)
    hs_keep = [0]
    for i in range(1, len(hs_t)):
        if (hs_t[i] - hs_t[hs_keep[-1]]) >= min_step_s:
            hs_keep.append(i)

    hs_t2 = hs_t[hs_keep]
    to_t2 = to_t[hs_keep]  # mantém pareado

    return hs_t2, to_t2


# ============================================================
# 6) Métricas simples (tempo)
# ============================================================
def cadence_from_hs(hs_t):
    """
    Cadência (spm) baseada em HS consecutivos do mesmo pé:
      step_time = diff(HS)
      cadence = 60 / mean(step_time)
    """
    hs_t = np.asarray(hs_t, float)
    if len(hs_t) < 3:
        return np.nan, np.nan, np.nan
    dt = np.diff(hs_t)
    cad = 60.0 / np.mean(dt)
    cv = 100.0 * np.std(dt, ddof=1) / np.mean(dt) if len(dt) >= 2 else np.nan
    return float(cad), float(np.mean(dt)), float(cv)

def stance_swing_times(hs_t, to_t):
    """
    stance = TO - HS
    swing  = próximo HS - TO
    """
    hs_t = np.asarray(hs_t, float)
    to_t = np.asarray(to_t, float)
    if len(hs_t) < 3:
        return np.nan, np.nan

    stance = to_t - hs_t
    swing = hs_t[1:] - to_t[:-1]

    return float(np.nanmean(stance)), float(np.nanmean(swing))


# ============================================================
# 7) Rodar tudo (FP1 e FP2) + tabela
# ============================================================
def run_trial(txt_path,
              thr_N=40.0,
              smooth_s=0.02,
              min_contact_s=0.08,
              min_gap_s=0.25,
              min_step_s=0.35):
    df, t = read_track_omc_txt(txt_path, sep="\t")

    # força vertical
    fz1 = pd.to_numeric(df["FP1.ForZ"], errors="coerce").to_numpy()
    fz2 = pd.to_numeric(df["FP2.ForZ"], errors="coerce").to_numpy()

    # eventos por plataforma
    hs1_i, to1_i, hs1_t, to1_t, fz1_s = detect_hs_to_from_fz(
        t, fz1, thr_N=thr_N, smooth_s=smooth_s, min_contact_s=min_contact_s
    )
    hs2_i, to2_i, hs2_t, to2_t, fz2_s = detect_hs_to_from_fz(
        t, fz2, thr_N=thr_N, smooth_s=smooth_s, min_contact_s=min_contact_s
    )

    # limpeza global
    hs1_t, to1_t = clean_events_global(t, hs1_t, to1_t, min_step_s=min_step_s)
    hs2_t, to2_t = clean_events_global(t, hs2_t, to2_t, min_step_s=min_step_s)

    # gaps
    gaps, gap_mask = detect_no_contact_gaps(
        t, fz1_s, fz2_s, thr_N=thr_N, min_gap_s=min_gap_s
    )

    # métricas (por plataforma/pé)
    cad1, step1, cv1 = cadence_from_hs(hs1_t)
    cad2, step2, cv2 = cadence_from_hs(hs2_t)

    stance1, swing1 = stance_swing_times(hs1_t, to1_t)
    stance2, swing2 = stance_swing_times(hs2_t, to2_t)

    summary = pd.DataFrame([
        {"foot/platform": "FP1", "n_HS": len(hs1_t), "cadence_spm": cad1, "mean_step_s": step1, "cv_step_%": cv1,
         "mean_stance_s": stance1, "mean_swing_s": swing1},
        {"foot/platform": "FP2", "n_HS": len(hs2_t), "cadence_spm": cad2, "mean_step_s": step2, "cv_step_%": cv2,
         "mean_stance_s": stance2, "mean_swing_s": swing2},
        {"foot/platform": "GAPS", "n_HS": len(gaps), "cadence_spm": np.nan, "mean_step_s": np.nan, "cv_step_%": np.nan,
         "mean_stance_s": np.nan, "mean_swing_s": np.nan},
    ])

    pack = {
        "df": df,
        "t": t,
        "fz1_raw": fz1, "fz2_raw": fz2,
        "fz1_s": fz1_s, "fz2_s": fz2_s,
        "hs1_t": hs1_t, "to1_t": to1_t,
        "hs2_t": hs2_t, "to2_t": to2_t,
        "gaps": gaps,
        "gap_mask": gap_mask,
        "thr_N": thr_N
    }

    return summary, pack


# ============================================================
# 8) Plot: ForZ + HS/TO + gaps
# ============================================================
def plot_fz_events(pack, tmin=0, tmax=20):
    t = pack["t"]
    fz1 = pack["fz1_s"]
    fz2 = pack["fz2_s"]
    thr = pack["thr_N"]

    mask = (t >= tmin) & (t <= tmax)

    plt.figure(figsize=(12,5))
    plt.plot(t[mask], fz1[mask], label="FP1.ForZ (suavizado)")
    plt.plot(t[mask], fz2[mask], label="FP2.ForZ (suavizado)")
    plt.axhline(thr, ls="--")

    # HS/TO
    for x in pack["hs1_t"]:
        if tmin <= x <= tmax:
            plt.axvline(x, alpha=0.35)
    for x in pack["to1_t"]:
        if tmin <= x <= tmax:
            plt.axvline(x, alpha=0.20)

    for x in pack["hs2_t"]:
        if tmin <= x <= tmax:
            plt.axvline(x, alpha=0.35)
    for x in pack["to2_t"]:
        if tmin <= x <= tmax:
            plt.axvline(x, alpha=0.20)

    # gaps (sombras)
    for (a,b) in pack["gaps"]:
        if b < tmin or a > tmax:
            continue
        aa, bb = max(a,tmin), min(b,tmax)
        plt.axvspan(aa, bb, alpha=0.15)

    plt.xlabel("Tempo (s)")
    plt.ylabel("Força vertical (N)")
    plt.title("ForZ (FP1/FP2) + HS/TO (linhas) + gaps (faixas)")
    plt.legend()
    plt.tight_layout()
    plt.show()


# ============================================================
# 9) EXEMPLO DE USO (ajuste o caminho)
# ============================================================
# txt_path = "/content/drive/MyDrive/.../Track_OMC_0001.txt"
# summary, pack = run_trial(txt_path, thr_N=40, smooth_s=0.02, min_contact_s=0.08, min_gap_s=0.25, min_step_s=0.35)
# display(summary)
# plot_fz_events(pack, tmin=0, tmax=20)
